In [1]:
import os
import pickle
import time
from pprint import pprint
from typing import List

import datasets
import kscope
from datasets import Dataset
from kscope import Model

/usr/local/anaconda3/envs/prompt_fine_tune_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Establish a client connection to the kscope service
client = kscope.Client(gateway_host="llm.cluster.local", gateway_port=3001)
client.model_instances

[{'id': 'b11f3264-9c03-4114-9d56-d39a0fa63640',
  'name': 'OPT-175B',
  'state': 'ACTIVE'}]

In [3]:
model = client.load_model("OPT-175B")
# If this model is not actively running, it will get launched in the background.
# In this case, wait until it moves into an "ACTIVE" state before proceeding.
while model.state != "ACTIVE":
    time.sleep(1)

We need to configure the model to generate in the way we want it to. However, because we only care about the activations of our input, the configuration is less important. We need one but the parameters don"t really matter. If you"re curious about the values, please see other 

In [4]:
short_generation_config = {"max_tokens": 1, "top_k": 4, "top_p": 3, "rep_penalty": 1.0, "temperature": 1.0}

### Activation Generation 

Activation generation is quite easy. We can use the client to query the remote model and explore the various modules.

In [5]:
model.module_names

['decoder',
 'decoder.embed_tokens',
 'decoder.embed_positions',
 'decoder.layers',
 'decoder.layers.0',
 'decoder.layers.0.dropout_module',
 'decoder.layers.0.self_attn',
 'decoder.layers.0.self_attn.dropout_module',
 'decoder.layers.0.self_attn.qkv_proj',
 'decoder.layers.0.self_attn.out_proj',
 'decoder.layers.0.self_attn_layer_norm',
 'decoder.layers.0.fc1',
 'decoder.layers.0.fc2',
 'decoder.layers.0.final_layer_norm',
 'decoder.layers.1',
 'decoder.layers.1.dropout_module',
 'decoder.layers.1.self_attn',
 'decoder.layers.1.self_attn.dropout_module',
 'decoder.layers.1.self_attn.qkv_proj',
 'decoder.layers.1.self_attn.out_proj',
 'decoder.layers.1.self_attn_layer_norm',
 'decoder.layers.1.fc1',
 'decoder.layers.1.fc2',
 'decoder.layers.1.final_layer_norm',
 'decoder.layers.2',
 'decoder.layers.2.dropout_module',
 'decoder.layers.2.self_attn',
 'decoder.layers.2.self_attn.dropout_module',
 'decoder.layers.2.self_attn.qkv_proj',
 'decoder.layers.2.self_attn.out_proj',
 'decoder.laye

We can select the module names of interest and pass them into a `get_activations` function alongside our set of prompts.

In [6]:
prompts = ["Hello World", "Fizz Buzz"]

module_name = "decoder.layers.95.fc2"

activations = model.get_activations(prompts, [module_name], short_generation_config)
pprint(activations)

# We sent a batch of 2 prompts to the model. So there is a list of length two activations returned
for activations_single_prompt in activations.activations:
    # For each prompt we extract the activations and calculate which label had the high likelihood.
    raw_activations = activations_single_prompt[module_name]
    # The activations should have shape (number of tokens) x (activation size)
    # For example, OPT-175 has an embedding dimension for the layer requested of 12288
    print("Tensor Shape:", raw_activations.shape)

Activations(activations=[{'decoder.layers.95.fc2': tensor([[ 0.0062, -0.4624,  0.4702,  ...,  0.0985,  1.2588, -0.7002],
        [-0.2197,  0.3364, -0.1965,  ..., -0.2651,  0.0475, -0.1638]],
       dtype=torch.float16)}, {'decoder.layers.95.fc2': tensor([[ 2.5312, -1.4609, -1.7266,  ...,  1.9453, -0.8262, -1.5234],
        [-0.2294, -0.6396,  0.9043,  ..., -0.8140,  0.1183,  0.2437],
        [-0.8062,  0.8320, -0.8003,  ..., -0.8530, -0.7656,  0.5718]],
       dtype=torch.float16)}], logprobs=[[None, -7.15677547454834, -6.544065475463867], [None, -5.25247859954834, -6.673819065093994, -5.898566246032715]], text=['Hello World', 'Fizz Buzz'], tokens=[['</s>', 'Hello', ' World'], ['</s>', 'F', 'izz', ' Buzz']])
Tensor Shape: torch.Size([2, 12288])
Tensor Shape: torch.Size([3, 12288])


As a proof of concept of the few-shot abilities of LLMs, we"ll only use a small training dataset and will only perform validation using a small test subset for compute efficiency.

* Training set: 100 randomly sampled training examples
* Test set: 300 randomly sample test examples

In [7]:
imdb = datasets.load_dataset("imdb")
train_size = 100
test_size = 300
n_demonstrations = 5

activation_save_path = "./resources/"

small_train_dataset = imdb["train"].shuffle(seed=42).select([i for i in list(range(train_size))])
small_test_dataset = imdb["test"].shuffle(seed=42).select([i for i in list(range(test_size))])
# We"re going to be experimenting with the affect that prompting the model for the task we envision affects the
# classifiers downstream performance. So we construct demonstrations here.
small_demonstration_set = imdb["train"].shuffle(seed=42).select([i for i in list(range(n_demonstrations))])

Found cached dataset imdb (/Users/david/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0)
100%|██████████| 3/3 [00:00<00:00, 244.08it/s]
Loading cached shuffled indices for dataset at /Users/david/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0/cache-9c48ce5d173413c7.arrow
Loading cached shuffled indices for dataset at /Users/david/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0/cache-c1eaa46e94dfbfd3.arrow
Loading cached shuffled indices for dataset at /Users/david/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0/cache-9c48ce5d173413c7.arrow


In [8]:
def batcher(seq: List[str], size: int) -> Dataset:
    return (seq[pos : pos + size] for pos in range(0, len(seq), size))

Let"s start by getting the activations associated with the raw review text. We"ll do activations for the text coupled with a prompt below

In [9]:
def generate_dataset_activations(
    split: str,
    inputs: List[str],
    labels: List[int],
    model: Model,
    module_name: str,
    pickle_name: str,
    batch_size: int = 16,
) -> None:
    print("Generating Activations with Prompts: " + split)

    activations = []
    for batch_number, input_batch in enumerate(batcher(inputs, batch_size)):
        activations.append(model.get_activations(input_batch, [module_name], short_generation_config))
        print(f"Batch Number {batch_number} Complete")

    parsed_activations = []
    for batch in activations:
        for input_activation in batch.activations:
            # We will be performing classification on the last token non-pad token of the sequence
            # common practice for autoregressive models (e.g. GPT-3). So we only keep the last row of the activation
            # outputs
            parsed_activations.append(input_activation[module_name][-1].float())

    cached_activations = {"activations": parsed_activations, "labels": labels}

    with open(os.path.join(activation_save_path, f"{split}{pickle_name}.pkl"), "wb") as handle:
        pickle.dump(cached_activations, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [10]:
module_name = "decoder.layers.95.fc2"

train_labels = small_train_dataset["label"]
test_labels = small_test_dataset["label"]

generate_dataset_activations(
    "train", small_train_dataset["text"], train_labels, model, module_name, "_activations_demo"
)
generate_dataset_activations("test", small_test_dataset["text"], test_labels, model, module_name, "_activations_demo")

Generating Activations with Prompts: train
Batch Number 0 Complete
Batch Number 1 Complete
Batch Number 2 Complete
Batch Number 3 Complete
Batch Number 4 Complete
Batch Number 5 Complete
Batch Number 6 Complete
Generating Activations with Prompts: test
Batch Number 0 Complete
Batch Number 1 Complete
Batch Number 2 Complete
Batch Number 3 Complete
Batch Number 4 Complete
Batch Number 5 Complete
Batch Number 6 Complete
Batch Number 7 Complete
Batch Number 8 Complete
Batch Number 9 Complete
Batch Number 10 Complete
Batch Number 11 Complete
Batch Number 12 Complete
Batch Number 13 Complete
Batch Number 14 Complete
Batch Number 15 Complete
Batch Number 16 Complete
Batch Number 17 Complete
Batch Number 18 Complete


Now let"s generate activations pre-conditioned with an instruction and a few demonstrations.

In [11]:
def create_demonstrations(instruction: str, demonstration_set: Dataset) -> str:
    label_int_to_str = {0: "negative", 1: "positive"}
    demonstration = f"{instruction}"
    demo_texts = demonstration_set["text"]
    demo_labels = demonstration_set["label"]
    for text, label in zip(demo_texts, demo_labels):
        # truncate the text in case it is very long
        split_text = text.split(" ")
        if len(split_text) > 128:
            text = " ".join(split_text[-128:])
        demonstration = f"{demonstration}\n\nText: {text} The sentiment is {label_int_to_str[label]}."
    return f"{demonstration}\n\n"

In [12]:
def create_prompts(texts: List[str], demonstration: str) -> List[str]:
    return [f"{demonstration}Text: {text} The sentiment is" for text in texts]

In [13]:
demonstration = create_demonstrations("Classify the sentiment of the text.", small_demonstration_set)
print(f"Demonstration:\n{demonstration}")

train_prompts = create_prompts(small_train_dataset["text"], demonstration)
test_prompts = create_prompts(small_test_dataset["text"], demonstration)
print(f"Prompt Example:\n{train_prompts[0]}")

train_labels = small_train_dataset["label"]
test_labels = small_test_dataset["label"]

Demonstration:
Classify the sentiment of the text.

Text: There is no relation at all between Fortier and Profiler but the fact that both are police series about violent crimes. Profiler looks crispy, Fortier looks classic. Profiler plots are quite simple. Fortier's plot are far more complicated... Fortier looks more like Prime Suspect, if we have to spot similarities... The main character is weak and weirdo, but have "clairvoyance". People like to compare, to judge, to evaluate. How about just enjoying? Funny thing too, people writing Fortier looks American but, on the other hand, arguing they prefer American series (!!!). Maybe it's the language, or the spirit, but I think this series is more English than American. By the way, the actors are really good and funny. The acting is not superficial at all... The sentiment is positive.

Text: a great. The plot is very true to the book which is a classic written by Mark Twain. The movie starts of with a scene where Hank sings a song with a 

In [14]:
module_name = "decoder.layers.95.fc2"
generate_dataset_activations(
    "train", train_prompts, train_labels, model, module_name, "_activations_with_prompts_demo"
)
generate_dataset_activations(
    "test",
    test_prompts,
    test_labels,
    model,
    module_name,
    "_activations_with_prompts_demo",
)

Generating Activations with Prompts: train
Batch Number 0 Complete
Batch Number 1 Complete
Batch Number 2 Complete
Batch Number 3 Complete
Batch Number 4 Complete
Batch Number 5 Complete
Batch Number 6 Complete
Generating Activations with Prompts: test
Batch Number 0 Complete
Batch Number 1 Complete
Batch Number 2 Complete
Batch Number 3 Complete
Batch Number 4 Complete
Batch Number 5 Complete
Batch Number 6 Complete
Batch Number 7 Complete
Batch Number 8 Complete
Batch Number 9 Complete
Batch Number 10 Complete
Batch Number 11 Complete
Batch Number 12 Complete
Batch Number 13 Complete
Batch Number 14 Complete
Batch Number 15 Complete
Batch Number 16 Complete
Batch Number 17 Complete
Batch Number 18 Complete


With these activations saved, the next step is to train a simple classifier on top of them in order to perform the sentiment classification. This is done in the `train_on_activations.ipynb` notebook.